<a href="https://colab.research.google.com/github/minju-Kang/MultiviewImageClassification/blob/main/Multiview_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
import glob
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import load_model

# MLP
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# dt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pydotplus
from sklearn.tree import export_graphviz
from IPython.core.display import Image

import matplotlib.pyplot as plt

# SVM
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import PolynomialFeatures

# rf
from sklearn.ensemble import RandomForestClassifier

In [ ]:
seed = 0
tf.random.set_seed(seed)

model_metrics = 'acc'    # acc, mse, mae (epoch마다 표시)
num_epochs = 10
num_batch = 100
num_split = 0.2

org_input_num = 8
train_path = r'/content/gdrive/My Drive/training set'
train_files = glob.glob(os.path.join(train_path, "*.csv"))
train_df_each_file = (pd.read_csv(f, header=None) for f in train_files)    # generator 생성

test_path = r'/content/gdrive/My Drive/test set'
test_files = glob.glob(os.path.join(test_path, "*.csv"))
test_df_each_file = (pd.read_csv(f, header=None) for f in test_files)    # generator 생성
test_df = pd.concat(test_df_each_file, ignore_index=True)
test_df = test_df.to_numpy()

In [ ]:
# class별 비율 확인
train_list = list(train_df_each_file)

ratio = 2
sampling_ratio = []

for i in train_list:
    class0 = len(i[i[org_input_num]==0])
    class1 = len(i[i[org_input_num]==1])
    print("class0: ", class0)
    print("class1: ", class1)
    print("class0/class1: ", '%0.2f' %(class0/class1))
    print("sampling ratio: ", '%0.2f' %(ratio*class1/class0), '\n')
    sampling_ratio.append(ratio*class1/class0)

class0:  518271
class1:  3459
class0/class1:  149.83
sampling ratio:  0.01 

class0:  487654
class1:  34076
class0/class1:  14.31
sampling ratio:  0.14 

class0:  188105
class1:  8311
class0/class1:  22.63
sampling ratio:  0.09 



In [ ]:
# subsampling 
train_df = pd.DataFrame()
for data, smp_ratio in  zip(train_list, sampling_ratio):
    df_class0 = data[data[org_input_num]==0].sample(frac=smp_ratio)
    print("0: ", len(df_class0))
    df_class1 = data[data[org_input_num]==1]
    print("1: ", len(df_class1))
    train_df = pd.concat([train_df, df_class0, df_class1], ignore_index = True)
    print("concat: ", len(train_df), '\n')
train_df = train_df.to_numpy()

0:  6918
1:  3459
concat:  10377 

0:  68152
1:  34076
concat:  112605 

0:  16622
1:  8311
concat:  137538 



In [ ]:
inp_num = 8
x = train_df[:, 0:inp_num]
y = train_df[:, inp_num]
X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, random_state=seed)

X_test = test_df[:, 0:inp_num]
Y_test = test_df[:, inp_num]

# MLP

In [ ]:
# MLP: 8x30x1 - 100 epochs
num_epochs = 100

start = time.time()  # 시작 시간 저장

model = Sequential()
model.add(Dense(30, input_dim=inp_num, activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=[model_metrics])

history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=num_epochs, batch_size=num_batch)
model.save('model_mlp_100.h5')

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

Epoch 1/100
1101/1101 [==============================] - 2s 1ms/step - loss: 0.1969 - acc: 0.6790 - val_loss: 0.1101 - val_acc: 0.8664
Epoch 2/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.1051 - acc: 0.8675 - val_loss: 0.1007 - val_acc: 0.8675
Epoch 3/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0960 - acc: 0.8734 - val_loss: 0.0944 - val_acc: 0.8738
Epoch 4/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0915 - acc: 0.8777 - val_loss: 0.0908 - val_acc: 0.8773
Epoch 5/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0886 - acc: 0.8797 - val_loss: 0.0892 - val_acc: 0.8777
Epoch 6/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0883 - acc: 0.8773 - val_loss: 0.0860 - val_acc: 0.8796
Epoch 7/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0849 - acc: 0.8806 - val_loss: 0.0852 - val_acc: 0.8806
Epoch 8/100
1101/1101 [==============================] 

In [ ]:
model = load_model('model_mlp_100.h5')
eval_model = model.evaluate(X_test, Y_test)[1]
print("\nTest Accuracy: %.4f" % eval_model)

del model

2967/2967 [==============================] - 3s 927us/step - loss: 0.0894 - acc: 0.8680

Test Accuracy: 0.8680


In [ ]:
# MLP: 8x30x1 - 1000 epochs
num_epochs = 1000

start = time.time()  # 시작 시간 저장

model = Sequential()
model.add(Dense(30, input_dim=inp_num, activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=[model_metrics])

history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=num_epochs, batch_size=num_batch)

eval_model = model.evaluate(X_test, Y_test)[1]
print("\nTest Accuracy: %.4f" % eval_model)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del model

Epoch 1/1000
1101/1101 [==============================] - 2s 1ms/step - loss: 0.1416 - acc: 0.8216 - val_loss: 0.0998 - val_acc: 0.8749
Epoch 2/1000
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0959 - acc: 0.8749 - val_loss: 0.0918 - val_acc: 0.8745
Epoch 3/1000
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0902 - acc: 0.8763 - val_loss: 0.0892 - val_acc: 0.8771
Epoch 4/1000
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0879 - acc: 0.8781 - val_loss: 0.0888 - val_acc: 0.8752
Epoch 5/1000
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0859 - acc: 0.8796 - val_loss: 0.0875 - val_acc: 0.8752
Epoch 6/1000
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0862 - acc: 0.8768 - val_loss: 0.0857 - val_acc: 0.8783
Epoch 7/1000
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0843 - acc: 0.8803 - val_loss: 0.0844 - val_acc: 0.8799
Epoch 8/1000
1101/1101 [========================

In [ ]:
# MLP: 6x30x1 - epoch 100
num_epochs = 100
X_train_sub = X_train[:, [0,1,2,5,6,7]]
X_val_sub = X_val[:, [0,1,2,5,6,7]]
X_test_sub = X_test[:, [0,1,2,5,6,7]]

start = time.time()  # 시작 시간 저장

model = Sequential()
model.add(Dense(30, input_dim=6, activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=[model_metrics])

history = model.fit(X_train_sub, Y_train, validation_data=(X_val_sub, Y_val), epochs=num_epochs, batch_size=num_batch)

eval_model = model.evaluate(X_test_sub, Y_test)[1]
print("\nTest Accuracy: %.4f" % eval_model)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del model

Epoch 1/100
1101/1101 [==============================] - 2s 1ms/step - loss: 0.1498 - acc: 0.8030 - val_loss: 0.1054 - val_acc: 0.8669
Epoch 2/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.1012 - acc: 0.8688 - val_loss: 0.0983 - val_acc: 0.8693
Epoch 3/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0947 - acc: 0.8735 - val_loss: 0.0944 - val_acc: 0.8733
Epoch 4/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0924 - acc: 0.8757 - val_loss: 0.0918 - val_acc: 0.8752
Epoch 5/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0888 - acc: 0.8803 - val_loss: 0.0886 - val_acc: 0.8773
Epoch 6/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0879 - acc: 0.8792 - val_loss: 0.0888 - val_acc: 0.8758
Epoch 7/100
1101/1101 [==============================] - 1s 1ms/step - loss: 0.0838 - acc: 0.8852 - val_loss: 0.0846 - val_acc: 0.8811
Epoch 8/100
1101/1101 [==============================] 

# DT

In [ ]:
# DT - depth 5

start = time.time() 

dt = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, random_state=seed)
dt.fit(X_train, Y_train)

pred_dt = dt.predict(X_test)

accuracy_dt = []
accuracy_dt.append(accuracy_score(Y_test, pred_dt))
print('Accuracy: %.4f' % accuracy_dt[0])

print("time :", time.time() - start)

print("\nfeature importance: \n{}".format(dt.feature_importances_))

Accuracy: 0.8773
time : 0.12002420425415039

feature importance: 
[4.04924830e-02 5.85112436e-02 8.59875300e-01 1.30995658e-04
 0.00000000e+00 9.22744705e-03 2.66010942e-02 5.16143692e-03]


In [ ]:
dot_data = export_graphviz(dt, out_file=None, filled=True, rounded=True, special_characters=True, class_names=["0","1"])
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('dt_depth5.png')

True

In [ ]:
del dt, pred_dt

In [ ]:
# DT - depth 7

start = time.time() 

dt = DecisionTreeClassifier(max_depth=7, min_samples_leaf=5, random_state=seed)
dt.fit(X_train, Y_train)

pred_dt = dt.predict(X_test)

print("time :", time.time() - start) 

accuracy_dt.append(accuracy_score(Y_test, pred_dt))
print('Accuracy: %.4f' % accuracy_dt[1])

print("\nfeature importance: \n{}".format(dt.feature_importances_))

del dt, pred_dt

time : 0.14480042457580566
Accuracy: 0.8911

feature importance: 
[4.48350370e-02 6.29449281e-02 8.44977340e-01 1.48505760e-03
 5.59672590e-04 1.10777418e-02 2.81943806e-02 5.92584218e-03]


In [ ]:
# DT - depth 10

start = time.time() 

dt = DecisionTreeClassifier(max_depth=10, min_samples_leaf=5, random_state=seed)
dt.fit(X_train, Y_train)

pred_dt = dt.predict(X_test)

print("time :", time.time() - start) 

accuracy_dt.append(accuracy_score(Y_test, pred_dt))
print('Accuracy: %.4f' % accuracy_dt[2])

print("\nfeature importance: \n{}".format(dt.feature_importances_))

del dt, pred_dt

time : 0.16819000244140625
Accuracy: 0.8921

feature importance: 
[0.04575942 0.06818105 0.82930072 0.00381962 0.0033776  0.01250929
 0.03036419 0.0066881 ]


In [ ]:
# DT - depth 15

start = time.time() 

dt = DecisionTreeClassifier(max_depth=15, min_samples_leaf=5, random_state=seed)
dt.fit(X_train, Y_train)

pred_dt = dt.predict(X_test)

print("time :", time.time() - start) 

accuracy_dt.append(accuracy_score(Y_test, pred_dt))
print('Accuracy: %.4f' % accuracy_dt[2])

print("\nfeature importance: \n{}".format(dt.feature_importances_))

del dt, pred_dt

time : 0.2108922004699707
Accuracy: 0.8921

feature importance: 
[0.04581213 0.08578025 0.80372467 0.00495331 0.00492502 0.01479712
 0.03221578 0.00779172]


In [ ]:
# DT - depth 20

start = time.time() 

dt = DecisionTreeClassifier(max_depth=20, min_samples_leaf=5, random_state=seed)
dt.fit(X_train, Y_train)

pred_dt = dt.predict(X_test)

print("time :", time.time() - start) 

accuracy_dt.append(accuracy_score(Y_test, pred_dt))
print('Accuracy: %.4f' % accuracy_dt[2])

print("\nfeature importance: \n{}".format(dt.feature_importances_))

del dt, pred_dt

time : 0.22741341590881348
Accuracy: 0.8921

feature importance: 
[0.0453305  0.09556088 0.79182673 0.00515959 0.00532866 0.01578414
 0.03262717 0.00838233]


# SVM

In [ ]:
# SVM - C=1

start = time.time()  # 시작 시간 저장

svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42))
                  ])
svm_clf.fit(X_train, Y_train)

pred_svm = svm_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_svm))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del svm_clf, pred_svm

Accuracy: 0.8627
time : 4.744473695755005


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# SVM - C=10

start = time.time()  # 시작 시간 저장

svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=10, loss="hinge", random_state=42))
                  ])
svm_clf.fit(X_train, Y_train)

pred_svm = svm_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_svm))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del svm_clf, pred_svm

Accuracy: 0.8622
time : 17.18485689163208


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# SVM - C=100

start = time.time()  # 시작 시간 저장

svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=100, loss="hinge", random_state=42))
                  ])
svm_clf.fit(X_train, Y_train)

pred_svm = svm_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_svm))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del svm_clf, pred_svm

Accuracy: 0.8560
time : 18.55437135696411


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# SVM with polynomial feature (degree=2)

start = time.time()  # 시작 시간 저장

svm_clf = Pipeline([
                    ("poly_features", PolynomialFeatures(degree=2)),
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42))
                  ])
svm_clf.fit(X_train, Y_train)

pred_svm = svm_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_svm))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del svm_clf, pred_svm

Accuracy: 0.8665
time : 41.826897859573364


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# SVM with polynomial feature (degree=3)

start = time.time()  # 시작 시간 저장

svm_clf = Pipeline([
                    ("poly_features", PolynomialFeatures(degree=3)),
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42))
                  ])
svm_clf.fit(X_train, Y_train)

pred_svm = svm_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_svm))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del svm_clf, pred_svm

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.8677
time : 118.91642808914185


In [ ]:
# SVM with kernel

start = time.time()  # 시작 시간 저장

svm_clf = SVC(kernel = "rbf", C = 1, gamma = 0.5)
svm_clf.fit(X_train, Y_train)

pred_svm = svm_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_svm))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

del svm_clf, pred_svm

Accuracy: 0.8729
time : 1014.4531855583191


# RF

In [ ]:
# rf - 10
rf_clf = RandomForestClassifier(n_estimators=10, random_state=42)
rf_clf.fit(X_train, Y_train)

pred_rf = rf_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_rf))

del rf_clf, pred_rf

Accuracy: 0.9104


In [ ]:
# rf - 50
rf_clf = RandomForestClassifier(n_estimators=50, random_state=42)
rf_clf.fit(X_train, Y_train)

pred_rf = rf_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_rf))

del rf_clf, pred_rf

Accuracy: 0.9140


In [ ]:
# rf - 100
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, Y_train)

pred_rf = rf_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_rf))

del rf_clf, pred_rf

Accuracy: 0.9144


In [ ]:
# rf - 200
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
rf_clf.fit(X_train, Y_train)

pred_rf = rf_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_rf))

del rf_clf, pred_rf

Accuracy: 0.9150


In [ ]:
# rf - 300
rf_clf = RandomForestClassifier(n_estimators=300, random_state=42)
rf_clf.fit(X_train, Y_train)

pred_rf = rf_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_rf))

del rf_clf, pred_rf

Accuracy: 0.9149


In [ ]:
# rf - 400
rf_clf = RandomForestClassifier(n_estimators=400, random_state=42)
rf_clf.fit(X_train, Y_train)

pred_rf = rf_clf.predict(X_test)
print('Accuracy: %.4f'  % accuracy_score(Y_test, pred_rf))

del rf_clf, pred_rf

Accuracy: 0.9145
